In [ ]:
import pandas as pd
import librosa
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import os

# Extract features from audio

In [ ]:
def interleave_arrays(arr1: np.ndarray, arr2: np.ndarray) -> np.ndarray:
    interleaved = np.empty(arr1.size + arr2.size, dtype=arr1.dtype)
    interleaved[0::2] = arr1
    interleaved[1::2] = arr2
    return interleaved

In [ ]:
def audio_to_dfcols(audio_filepath: str, df_cols: np.ndarray) -> pd.DataFrame:
  y, sr = librosa.load(audio_filepath)

  # duration = librosa.get_duration(y = y, sr = sr)
  chroma_stft_res = librosa.feature.chroma_stft(y = y, sr = sr)
  rms_res = librosa.feature.rms(y = y)
  spectral_centroid_res = librosa.feature.spectral_centroid(y = y, sr = sr)
  spectral_bandwidth_res = librosa.feature.spectral_bandwidth(y = y, sr = sr)
  spectral_rolloff_res = librosa.feature.spectral_rolloff(y = y, sr = sr)
  zero_crossing_rate_res = librosa.feature.zero_crossing_rate(y = y)
  harmonic_res = librosa.effects.harmonic(y)
  percussive_res = librosa.effects.percussive(y)
  tempo = librosa.beat.tempo(y = y, sr = sr)
  mfccs = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = 20)
  mfccs_means = np.mean(mfccs, axis = 1)
  mfccs_vars = np.var(mfccs, axis = 1)
  result: np.ndarray = np.concatenate((
    # np.array(duration),
   np.mean(chroma_stft_res), np.var(chroma_stft_res),
   np.mean(rms_res), np.var(rms_res),
   np.mean(spectral_centroid_res), np.var(spectral_centroid_res),
   np.mean(spectral_bandwidth_res), np.var(spectral_bandwidth_res),
   np.mean(spectral_rolloff_res), np.var(spectral_rolloff_res),
   np.mean(zero_crossing_rate_res), np.var(zero_crossing_rate_res),
   np.mean(harmonic_res), np.var(harmonic_res),
   np.mean(percussive_res), np.var(percussive_res),
   np.array(tempo)), axis = None)
  result = np.concatenate((result, interleave_arrays(mfccs_means, mfccs_vars)), axis = None)
  # print(result)
  # print(result.shape)
  # print(df_cols.shape)
  # return dict(zip(df_cols, result))
  # return pd.DataFrame(dict(zip(df_cols, result)))
  return pd.DataFrame(data = result.reshape(1, -1), columns = df_cols)

In [ ]:
def addNewMusic(audio_data_route: str, csv_data_route: str, genre: str):
    # Check if the CSV file exists
    if os.path.isfile(csv_data_route):
        # CSV file exists, read it
        df = pd.read_csv(csv_data_route)
    else:
        # CSV file doesn't exist, create an empty DataFrame
        df = pd.DataFrame(columns=['chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label'])

    # Generate new row using audio_to_dfcols
    audio_1_df = audio_to_dfcols(audio_data_route, df.columns[:-1].to_numpy())

    # Add the genre as the last entry
    audio_1_df['label'] = genre

    # Append new row to DataFrame
    df = pd.concat([df, audio_1_df], ignore_index=True)

    # Write DataFrame back to CSV file
    df.to_csv(csv_data_route, index=False)

In [ ]:
csv_data_route = 'feature.csv'
genre = 'rock'

# Loop through each file from blues.00000.wav to blues.00099.wav
for i in range(100):
    # Generate the filename
    audio_data_route = f'rock.{i:05d}.wav'

    # Call the addNewMusic function
    addNewMusic(audio_data_route, csv_data_route, genre)
